<a href="https://colab.research.google.com/github/ds-joy/ML-Practice/blob/master/deep_learning_with_python/chap_05/dog_vs_cat/dog_vs_cat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Loading data to google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
% cd /content/drive/My Drive/Colab Notebooks/cat_vs_dog


[Errno 2] No such file or directory: '/content/drive/My Drive/Colab Notebooks/cat_vs_dog'
/content


In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"dsjoy36","key":"ca378101f15734ddc9cce0bdee3926b1"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!rm kaggle.json

In [ ]:
!kaggle competitions download -c dogs-vs-cats

 98% 265M/271M [00:04<00:00, 80.9MB/s]
100% 271M/271M [00:04<00:00, 59.9MB/s]
  0% 0.00/86.8k [00:00<?, ?B/s]
100% 86.8k/86.8k [00:00<00:00, 89.4MB/s]
 98% 532M/543M [00:06<00:00, 120MB/s] 
100% 543M/543M [00:06<00:00, 89.8MB/s]


In [ ]:
!ls

drive  sample_data  sampleSubmission.csv  test1.zip  train.zip


In [ ]:
!unzip test1.zip
!unzip train.zip

In [ ]:
import os, shutil
print('total images in train folder: ', len(os.listdir('train')))


total images in train folder:  25000


#Importing and setting paths

###Imports

In [ ]:
import cv2
import numpy as np
import os
from random import shuffle
from tqdm import tqdm

###Paths

In [ ]:
TRAIN_DIR = '/content/drive/My Drive/Colab Notebooks/dog_vs_cat/train'
TEST_DIR = '/content/drive/My Drive/Colab Notebooks/dog_vs_cat/test1'
IMG_SIZE = 50
LR = 1e-3

MODEL_NAME = 'dogvscats-{}-{}.model'.format(LR, '2conv-besic' )

#Labeling the images

In [ ]:
def label_img(img):
    word_label = img.split('.')[-3]
    if word_label == 'cat': return [1, 0]
    elif word_label == 'dog': return [0, 1]

#Create training data

In [ ]:
def create_training_data():
    training_data = []

    for img in tqdm(os.listdir(TRAIN_DIR)):
        # calling the label_img function which returns
        # one hot array i.e [1, 0]
        label = label_img(img)
        path = os.path.join(TRAIN_DIR, img)
        img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE), (IMG_SIZE, IMG_SIZE) )
        training_data.append( [ np.array(img), np.array(label) ] )

    shuffle(training_data)
    np.save('train_data.npy', training_data)
    return training_data


#Creating testing data

In [ ]:
def process_test_data():
    testing_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
        path = os.path.join(TEST_DIR, img)
        img_num = img.split('.')[0]
        img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE), 
                         (IMG_SIZE, IMG_SIZE) )
        testing_data.append([np.array(img), img_num])
    np.save('test_data.npy', testing_data)
    return testing_data
    

In [ ]:
train_data = create_training_data()

  0%|          | 119/25000 [01:36<5:01:31,  1.38it/s]

#Neural network

In [ ]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')